In [1]:
import os
import getpass

if os.name == 'posix':
    password = getpass.getpass("Enter sudo password: ")
    os.system(f'echo {password} | sudo -S apt install tesseract-ocr poppler-utils -y')

[sudo] password for thienan: 



Reading package lists...
Building dependency tree...
Reading state information...

Reading state information...
tesseract-ocr is already the newest version (5.3.4-1build5).
poppler-utils is already the newest version (24.02.0-1ubuntu9.8).
The following packages were automatically installed and are no longer required:
  binutils-aarch64-linux-gnu cpp-13-aarch64-linux-gnu cpp-9-aarch64-linux-gnu
  cpp-aarch64-linux-gnu gcc-11-cross-base gcc-13-aarch64-linux-gnu-base
  gcc-9-aarch64-linux-gnu-base gcc-9-cross-base libasan5-arm64-cross
  libasan8-arm64-cross libatomic1-arm64-cross libgcc-13-dev-arm64-cross
  libgcc-9-dev-arm64-cross libgcc-s1-arm64-cross libgomp1-arm64-cross
  libhwasan0-arm64-cross libitm1-arm64-cross liblsan0-arm64-cross
  libstdc++6-arm64-cross libtsan0-arm64-cross libtsan2-arm64-cross
  libubsan1-arm64-cross
Use 'sudo apt autoremove' to remove them.
tesseract-ocr is already the newest version (5.3.4-1build5).
poppler-utils is already the newest version (24.02.0-1ubuntu

In [2]:
! pip install pdf2image pytesseract opencv-python python-dotenv zipfile36
! pip install paddlepaddle==3.2.0 -i https://www.paddlepaddle.org.cn/packages/stable/cpu/
! pip install paddleocr

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://www.paddlepaddle.o

In [1]:
from pdf2image import convert_from_path
import pytesseract
import os
import requests
import zipfile
import io
import json
from paddleocr import PaddleOCR
from dotenv import load_dotenv
from ocr.utils import (ocr_layout,
                   extract_vn_letters,
                   extract_cn_letters,)

/home/thienan/miniconda3/envs/mt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download Tessdata for **Tesseract**

In [14]:
def download_tessdata(lang):
    local_dir = os.path.join(os.getcwd(), 'tessdata')
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
        
    file_path = os.path.join(local_dir, f'{lang}.traineddata')
    
    # Check if file exists and is likely invalid (too small, e.g. HTML page)
    if os.path.exists(file_path) and os.path.getsize(file_path) < 1024 * 1024: # < 1MB
        print(f"Removing invalid file: {file_path}")
        os.remove(file_path)
    
    # Tải về nếu file chưa tồn tại
    if not os.path.exists(file_path):
        # Use 'raw' URL to get the actual file content, not the HTML page
        url = f"https://github.com/tesseract-ocr/tessdata/raw/main/{lang}.traineddata"
        print(f"Downloading {url}...")
        response = requests.get(url)
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Download tessdata for {lang} successfully")
    else:
        print(f"Tessdata for {lang} existed")
        
    return local_dir

In [16]:
tessdata_path = download_tessdata('vie')
tessdata_path = download_tessdata('chi-sim')

# Khi gọi tesseract để biết sẽ config folder testdata của mình tạo
custom_config = f'--tessdata-dir "{tessdata_path}"'

Tessdata for vie existed
Removing invalid file: /home/thienan/Documents/coding/zh-vn-mt/tessdata/chi-sim.traineddata
Download tessdata for chi-sim successfully
Download tessdata for chi-sim successfully


## Download Poppler for **pdf2image**

In [4]:
def install_poppler():
    local_dir = os.path.join(os.getcwd(), 'Poppler')
    bin_path = os.path.join(local_dir,'poppler-25.11.0', 'Library', 'bin')
    
    url = "https://github.com/oschwartz10612/poppler-windows/releases/download/v25.11.0-0/Release-25.11.0-0.zip"
    
    response = requests.get(url)
    response.raise_for_status()
    
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
        
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(local_dir)
        
    return bin_path

In [5]:
if os.name == 'nt':
    poppler_path = install_poppler()
else:
    poppler_path = None

## Run OCR

In [6]:
load_dotenv()

if os.name == 'nt':
    tesseract_path = os.getenv('TESSERACT_PATH')
    pytesseract.pytesseract.tesseract_cmd = tesseract_path

In [7]:
start_page = 107
end_page = 108

pages = convert_from_path('pdf1.pdf', 
                          poppler_path=poppler_path, 
                          dpi=400,
                          first_page = start_page,
                          last_page = end_page,
                          )

In [8]:
paddle_ocr = PaddleOCR(lang='ch', 
                use_doc_orientation_classify=False,
                use_doc_unwarping=False,
                use_textline_orientation=False,)

/home/thienan/miniconda3/envs/mt/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/thienan/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('PP-OCRv5_server_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/thienan/.paddlex/official_models/PP-OCRv5_server_rec`.


In [17]:
full_text = {}
full_text['CN'] = []
full_text['VN'] = []

for i, page_image in enumerate(pages):
    cn_text, vn_text = ocr_layout(page_image, paddle_ocr, i, tessdata_config=custom_config)
    full_text['CN'].append(cn_text)
    
    # vn_text sẽ chứa tiếng việt và tiếng pinyin -> sẽ xử lý sau
    full_text['VN'].append(vn_text)

In [18]:
letters_vi = []
for v in full_text['VN']:
    letters_vi.extend(extract_vn_letters(v))

In [19]:
letters_cn = []
for v in full_text['CN']:
    letters_cn.extend(extract_cn_letters(v))

In [ ]:
def save_to_json(filename):
    output_data = []

    for i, (vi_text, cn_text) in enumerate(zip(letters_vi, letters_cn), 1):
        item = {
            "src_id" : i,
            "src_lang" : vi_text,
            "tgt_lang" : cn_text
        }
        
        output_data.append(item)
        
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(
            output_data,
            f,
            indent = 4,
            ensure_ascii=False,
        )
        
    print("Save file json sucessfully")

In [21]:
save_to_json("ocr.json")

Save file json sucessfully
